In [ ]:
!mongoimport --db restaurants --collection restaurants --drop --file ./data/restaurants.json
!mongoimport --db restaurants --collection neighborhoods --drop --file ./data/neighborhoods.json

In [ ]:
import json
import pymongo

In [ ]:
HOST = "localhost"
PORT = "27017"
DB_NAME = "restaurants"

### Utilities functions

In [ ]:
def transform_json(element):
    geo_element = dict()
    geo_element["type"] = "Feature"
    geo_element["geometry"] = dict()
    
    try:
        geo_element["geometry"]["type"] = element["location"]["type"]
        geo_element["geometry"]["coordinates"] = element["location"]["coordinates"]
    except KeyError:
        geo_element["geometry"]["type"] = element["geometry"]["type"]
        geo_element["geometry"]["coordinates"] = element["geometry"]["coordinates"]
    
    geo_element["properties"] = dict()
    geo_element["properties"]["name"] = element["name"]
    return geo_element

In [ ]:
def save_json(data, filename = "ouput.json"):
    with open("./data/" + filename, "w") as outfile:
        json.dump(data, outfile, indent = 4, sort_keys = True)

In [ ]:
def create_feature_collection(features):  
    feature_collection = dict()
    feature_collection["type"] = "FeatureCollection"
    feature_collection["features"] = features
    return feature_collection

### Connecting MongoDB and setting up for geographic queries

In [ ]:
# Connecting to database
db = None
try:
    db_client = pymongo.MongoClient(HOST + ":" + PORT)
    db = db_client[DB_NAME]
    print("Conecction established successfully!")
except pymongo.errors.ConnectionFailure:
    print("Connection cannot be established")

In [ ]:
# Creating geospatial indexes: https://docs.mongodb.com/manual/core/2dsphere/
db.restaurants.create_index([("location", "2dsphere")])
db.neighborhoods.create_index([("geometry", "2dsphere")])

### Making some basic queries

In [ ]:
# Getting the first restaurant in restaurants collection
point = db.restaurants.find_one()
point

In [ ]:
# Transforming the previous result to a GeoJSON string
geo_point = transform_json(point)
geo_point

In [ ]:
# Storing GeoJSON string to disk
save_json(geo_point, "geo_point.json")

In [ ]:
# Getting the first neighborhood in neighborhoods collection
polygon = db.neighborhoods.find_one()

In [ ]:
# Transforming the previous result to a GeoJSON string
geo_polygon = transform_json(polygon)
geo_polygon

In [ ]:
# Storing GeoJSON string to disk
save_json(geo_polygon, "geo_polygon.json")

### Making geographical queries

In [ ]:
# Neighborhood intersects point
intersection = db.neighborhoods.find_one({"geometry": {"$geoIntersects": {"$geometry": {"type": "Point", "coordinates": [-73.93414657, 40.82302903]}}}})

In [ ]:
save_json(transform_json(intersection), "geo_intersection.json")

In [ ]:
# Restaurants within neighborhood
neighborhood = db.neighborhoods.find_one({"geometry": {"$geoIntersects": {"$geometry": {"type": "Point", "coordinates": [-73.93414657, 40.82302903]}}}})
restaurants = db.restaurants.find({"location": {"$geoWithin": {"$geometry": neighborhood["geometry"]}}})

geo_restaurants = []
for restaurant in restaurants:
    geo_restaurants.append(transform_json(restaurant))
geo_feature_collection = create_feature_collection(geo_restaurants)

In [ ]:
save_json(geo_feature_collection, "geo_feature_collection.json")

In [ ]:
# Restaurants by proximity
restaurants = db.restaurants.find({"location": {"$geoWithin": {"$centerSphere": [[-73.93414657, 40.82302903 ], 1.5/3963.2]}}}) # 1.5 miles

geo_restaurants = []
for restaurant in restaurants:
    geo_restaurants.append(transform_json( restaurant))
save_json( create_feature_collection(geo_restaurants), "geo_proximity.json")

In [ ]:
# Restaurants within sphere
restaurants = db.restaurants.find({"location": {"$nearSphere": {"$geometry": {"type": "Point", "coordinates": [-73.93414657, 40.82302903]}, "$minDistance": 1000, "$maxDistance": 1500}}})

geo_restaurants = []
for restaurant in restaurants:
    geo_restaurants.append(transform_json(restaurant))
save_json(create_feature_collection(geo_restaurants), "geo_sphere.json")